**Import libraries**

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.special import factorial
import h5py
import os
import scipy.io as sio
from google.colab import drive

**Connect dataset from Google Drive**

In [ ]:
drive.mount('/content/drive')
path = "drive/MyDrive/CS230_Project/Jenkins_Rstruct_Data"
!ls $path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
JR_2015-12-04_truncated2.mat  plan_test_data.mat  plan_training_data.mat



## Dataset information

In [ ]:
R = sio.loadmat(path+"/JR_2015-12-04_truncated2.mat")["R"][0]
ntrials = len(R)
shape = R[0]['spikeRaster'].todense().shape
fields = R[0].dtype.names

print("There are %d trials in the R-struct" % ntrials)
print("There are %d electrodes with %d millseconds of data \n" % (shape[0], shape[1]))
print("Fields in this dataset include: ")
for field in fields:
    print("-", field)

There are 506 trials in the R-struct
There are 96 electrodes with 901 millseconds of data 

Fields in this dataset include: 
- startDateNum
- startDateStr
- timeTargetOn
- timeTargetAcquire
- timeTargetHeld
- timeTrialEnd
- subject
- counter
- state
- cursorPos
- spikeRaster
- spikeRaster2
- isSuccessful
- trialNum
- timeFirstTargetAcquire
- timeLastTargetAcquire
- trialLength
- target


The fields we care about the most are `target` and `spikeRaster`.

The `target` field holds the coordinates of the reach direction of a given trial.
```python
# returns a tuple indicating the x, y, and z coordinates
# of the target reach direction for trial 0
R[0]['target']
```
The `spikeRaster` field holds a sparse matrix where each row corresponds to an electrode, and each column corresponds to a spike time. We will use the `.todense()` function to convert the row to an array of 1s and 0s where each index is a millisecond indicating whether a neuron fired (1) or not (0).
```python
# returns an array of spikes for the first trial and
# and the first electrode
R[0]['spikeRaster'].todense()[0, :]
# returns whether there was a spike for the first trial and
# and the first electrode during the 10th millisecond
R[0]['spikeRaster'].todense()[0,9]
```

There are 9 possible reach directions.
```python
# Targets sorted in CCW
xy_sorted = np.array([
             [0.0, 0.0],          # 0
             [120.0, 0.0],          # 1
             [84.85, 84.85],        # 2
             [0.0,   120.0],        # 3
             [-84.85,84.85],        # 4
             [-120.0, 0],           # 5
             [-84.85, -84.85],      # 6
             [0.0, -120],           # 7
             [84.85,-84.85]])       # 8
```

## Restructure the data

In [ ]:
import tensorflow as tf

In [ ]:
targets = R[1:]['target']
len(targets)

505

In [ ]:
num_electrodes = 96
ms = 400
ms_start = 200
START_SIZE = num_electrodes * ms
# xy_sorted = np.array([
#              [0.0, 0.0],
#              [120.0, 0.0],
#              [84.85, 84.85],
#              [0.0, 120.0],
#              [-84.85,84.85],
#              [-120.0, 0],
#              [-84.85, -84.85],
#              [0.0, -120],
#              [84.85,-84.85]])
xy_sorted = np.array([
             [120.0, 0.0],
             [84.85, 84.85],
             [0.0, 120.0],
             [-84.85,84.85],
             [-120.0, 0],
             [-84.85, -84.85],
             [0.0, -120],
             [84.85,-84.85]])

In [ ]:
middle_reach_dict = {
    0.0: 0.0,
    -84.85: 84.85,
    84.85: -84.85,
    -120.0: 120.0,
    120.0: -120.0,
}

In [ ]:
n_reach_trials = 505 # reaches not including the first
# n_reach_trials = 253 # reaches besides middle
spike_times = np.ndarray(shape=(n_reach_trials, num_electrodes, ms))
targets = np.ndarray(shape=(n_reach_trials, 1), dtype=int)
count = {}
per_reach = {}
total = 0
prev = (round(R[0]['target'][0].item(), 2), round(R[0]['target'][1].item(), 2))
for n in range(ntrials):
    spike_time = R[n]['spikeRaster'].todense()[:, ms_start: ms_start + ms]
    target_x = round(R[n]['target'][0].item(), 2)
    target_y = round(R[n]['target'][1].item(), 2)
    # first check if middle
    if target_x == round(0.0, 2) and target_y == round(0.0, 2):
        target_x = round(middle_reach_dict[prev[0]], 2)
        target_y = round(middle_reach_dict[prev[1]], 2)
    for d, dir in enumerate(xy_sorted):
        if target_x == round(dir[0], 2) and target_y == round(dir[1], 2):
            prev = (target_x, target_y)
            if d not in count:
                count[d] = 0
                per_reach[d] = []
            per_reach[d].append(spike_time)
            count[d] += 1
            # col[d] = 1
            targets[total, :] = d
            spike_times[total, :, :] = spike_time
            total += 1
            break
spike_times.shape, targets.shape, total

((505, 96, 400), (505, 1), 505)

In [ ]:
count # check distribution of data

{6: 63, 2: 63, 5: 63, 1: 62, 0: 64, 4: 64, 3: 63, 7: 63}

In [ ]:
train_size = .8
total = 0
for c in count:
    total += int(train_size * count[c])
total

401

### Evenly Sample From Each Class

In [ ]:
import sklearn.model_selection as sk

# num_train = 173
num_train = total
# num_train = int(n_reach_trials * .7)
X_train = np.ndarray(shape=(num_train, num_electrodes, ms))
X_test = np.ndarray(shape=(n_reach_trials - num_train, num_electrodes, ms))

Y_train = np.ndarray(shape=(num_train, 1), dtype=int)
Y_test = np.ndarray(shape=(n_reach_trials - num_train, 1), dtype=int)
train_start = 0
test_start = 0
for d in per_reach:
    reach = np.array(per_reach[d])
    target = np.full((reach.shape[0], 1), d)
    reach_x_train, reach_x_test, reach_y_train, reach_y_test = sk.train_test_split(reach, target, train_size=train_size, random_state = 42)
    print(reach_x_train.shape[0], reach_x_test.shape[0])
    X_train[train_start: train_start + reach_x_train.shape[0], :, :] = reach_x_train
    X_test[test_start: test_start + reach_x_test.shape[0], :, :] = reach_x_test
    Y_train[train_start: train_start + reach_y_train.shape[0], :] = reach_y_train
    Y_test[test_start: test_start + reach_y_test.shape[0], :] = reach_y_test

    train_start += reach_x_train.shape[0]
    test_start += reach_x_test.shape[0]


50 13
50 13
50 13
49 13
51 13
51 13
50 13
50 13


### Convert Data to `tf.data.Dataset`

In [ ]:
X_train = tf.cast(X_train, tf.float32)
X_test = tf.cast(X_test, tf.float32)
X_train2 = tf.data.Dataset.from_tensor_slices(X_train)
X_test2 = tf.data.Dataset.from_tensor_slices(X_test)
Y_train2 = tf.data.Dataset.from_tensor_slices(Y_train)
Y_test2 = tf.data.Dataset.from_tensor_slices(Y_test)

### Apply Map Functions to Data

In [ ]:
def one_hot_matrix(label, depth=8):
    """
    Computes the one hot encoding for a single label
    
    Arguments:
        label --  (int) Categorical labels
        depth --  (int) Number of different classes that label can take
    
    Returns:
         one_hot -- tf.Tensor A single-column matrix with the one hot encoding.
    """

    one_hot = tf.reshape(tf.one_hot(label, depth, axis=0), (depth,))
    return one_hot

def normalize(x):
    """
    Transform an image into a tensor of shape (96 * 300, )
    and normalize its components.
    
    Arguments
    image - Tensor.
    
    Returns: 
    result -- Transformed tensor 
    """
    x = tf.reshape(x, [-1,])
    return x

In [ ]:
x_train = X_train2.map(normalize) # flatten signals
x_test = X_test2.map(normalize)

In [ ]:
y_train = Y_train2.map(one_hot_matrix) # convert classes to one hot
y_test = Y_test2.map(one_hot_matrix)

In [ ]:
len(y_train), len(x_train)

(401, 401)

In [ ]:
print(next(iter(y_test)))

tf.Tensor([0. 0. 0. 0. 0. 0. 1. 0.], shape=(8,), dtype=float32)


## Create Model

### Helper Functions

In [ ]:
def initialize_parameters():
    """
    Initializes parameters to build a neural network with TensorFlow. The shapes are:
                        W1 : [40, 28800]
                        b1 : [40, 1]
                        W2 : [12, 40]
                        b2 : [12, 1]
                        W3 : [8, 12]
                        b3 : [8, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2, W3, b3
    """
                                
    initializer = tf.keras.initializers.GlorotNormal(seed=1)   

    W1 = tf.Variable(initializer(shape=(40, START_SIZE)))
    b1 = tf.Variable(initializer(shape=(40, 1)))
    W2 = tf.Variable(initializer(shape=(12, 40)))
    b2 = tf.Variable(initializer(shape=(12, 1)))
    W3 = tf.Variable(initializer(shape=(8, 12)))
    b3 = tf.Variable(initializer(shape=(8, 1)))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    Z1 = tf.math.add(tf.linalg.matmul(W1,  X), b1)
    A1 = tf.keras.activations.relu(Z1)
    Z2 = tf.math.add(tf.linalg.matmul(W2, A1), b2)
    A2 = tf.keras.activations.relu(Z2)
    Z3 = tf.math.add(tf.linalg.matmul(W3, A2), b3)

    
    return Z3

def compute_cost(logits, labels):
    """
    Computes the cost
    
    Arguments:
    logits -- output of forward propagation (output of the last LINEAR unit), of shape (8, num_examples)
    labels -- "true" labels vector, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    logits = tf.transpose(logits)
    labels = tf.transpose(labels)
    cost = tf.math.reduce_sum(tf.keras.losses.categorical_crossentropy(labels, logits, from_logits=True))
    # cost = tf.math.reduce_sum(tf.keras.losses.binary_crossentropy(labels, logits, from_logits=True))
    return cost

### Model

In [ ]:
from keras.optimizers import SGD

def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 4, print_cost = True):
    
    costs = []                                        # To keep track of the cost
    train_acc = []
    test_acc = []
  
    parameters = initialize_parameters()

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']

    optimizer = tf.keras.optimizers.Adam(learning_rate)
    #f1 score + confusion metrics
    
    # The CategoricalAccuracy will track the accuracy for this multiclass problem
    test_accuracy = tf.keras.metrics.CategoricalAccuracy()
    train_accuracy = tf.keras.metrics.CategoricalAccuracy()
    print(len(X_train), len(Y_train), X_train, Y_train)
    dataset = tf.data.Dataset.zip((X_train, Y_train))
    test_dataset = tf.data.Dataset.zip((X_test, Y_test))
    
    m = dataset.cardinality().numpy()
    
    minibatches = dataset.batch(minibatch_size).prefetch(8)
    test_minibatches = test_dataset.batch(minibatch_size).prefetch(8)
    
    for epoch in range(num_epochs):

        epoch_cost = 0.
        
        train_accuracy.reset_states()
        
        for (minibatch_X, minibatch_Y) in minibatches:
            
            with tf.GradientTape() as tape:
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)
                minibatch_cost = compute_cost(Z3, tf.transpose(minibatch_Y))

            train_accuracy.update_state(minibatch_Y, tf.transpose(Z3))
            
            trainable_variables = [W1, b1, W2, b2, W3, b3]
            grads = tape.gradient(minibatch_cost, trainable_variables)
            optimizer.apply_gradients(zip(grads, trainable_variables))
            epoch_cost += minibatch_cost
        
        epoch_cost /= m

        if print_cost == True and epoch % 10 == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            print("Train accuracy:", train_accuracy.result())
            
            for (minibatch_X, minibatch_Y) in test_minibatches:
                Z3 = forward_propagation(tf.transpose(minibatch_X), parameters)
                test_accuracy.update_state(minibatch_Y, tf.transpose(Z3))
            print("Test_accuracy:", test_accuracy.result())

            costs.append(epoch_cost)
            train_acc.append(train_accuracy.result())
            test_acc.append(test_accuracy.result())
            test_accuracy.reset_states()


    return parameters, costs, train_acc, test_acc

## Train Model

In [ ]:
parameters, costs, train_acc, test_acc = model(x_train, y_train, x_test, y_test, num_epochs=110)

401 401 <MapDataset element_spec=TensorSpec(shape=(28800,), dtype=tf.float32, name=None)> <MapDataset element_spec=TensorSpec(shape=(8,), dtype=tf.float32, name=None)>
Cost after epoch 0: 2.228898
Train accuracy: tf.Tensor(0.13965087, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.13461539, shape=(), dtype=float32)
Cost after epoch 10: 0.304657
Train accuracy: tf.Tensor(0.9725686, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.21153846, shape=(), dtype=float32)
Cost after epoch 20: 0.032504
Train accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.21153846, shape=(), dtype=float32)
Cost after epoch 30: 0.008805
Train accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.20192307, shape=(), dtype=float32)
Cost after epoch 40: 0.003429
Train accuracy: tf.Tensor(1.0, shape=(), dtype=float32)
Test_accuracy: tf.Tensor(0.21153846, shape=(), dtype=float32)
Cost after epoch 50: 0.001497
Train accuracy: tf.Tensor(1.0, shape=(), dtype=flo

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import tensorflow
np.random.seed(0)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
np.random.seed(1)

In [ ]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: Emojify_V2

def Neuro(input_shape):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    print(input_shape)
    input = Input(input_shape, dtype='float32')
    print(input.shape)
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(input)
    # Add dropout with a probability of 0.5
    X = Dropout(.7)(X) 
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(64, return_sequences=False)(X)
    # # # Add dropout with a probability of 0.5
    # X = Dropout(.7)(X)
    # X = LSTM(32, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(.7)(X)  
    # Propagate X through a Dense layer with 5 units
    X = Dense(8)(X) 
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(input, X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
model = Neuro((ms, num_electrodes))
model.summary()

(400, 96)
(None, 400, 96)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 400, 96)]         0         
                                                                 
 lstm_4 (LSTM)               (None, 400, 128)          115200    
                                                                 
 dropout_4 (Dropout)         (None, 400, 128)          0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 activation_2 (Activation)   (Non

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
Y_train.shape

(401, 1)

In [ ]:
Y_train_LSTM = []
for t in range(Y_train.shape[0]):
    col = np.zeros((8,))
    col[Y_train[t][0]] = 1
    Y_train_LSTM.append(col)
Y_train_LSTM = np.array(Y_train_LSTM)

Y_test_LSTM = []
for t in range(Y_test.shape[0]):
    col = np.zeros((8,))
    col[Y_test[t][0]] = 1
    Y_test_LSTM.append(col)
Y_test_LSTM = np.array(Y_test_LSTM)

Y_train_LSTM.shape, Y_test_LSTM.shape

((401, 8), (104, 8))

In [ ]:
X_train_LSTM = tf.reshape(X_train, (X_train.shape[0], X_train.shape[2], X_train.shape[1]))
X_test_LSTM = tf.reshape(X_test, (X_test.shape[0], X_test.shape[2], X_test.shape[1]))

In [ ]:
X_train_LSTM.shape, Y_train_LSTM.shape, X_test_LSTM.shape, Y_test_LSTM.shape

(TensorShape([401, 400, 96]), (401, 8), TensorShape([104, 400, 96]), (104, 8))

In [ ]:
X_train_LSTM[0][0]

<tf.Tensor: shape=(96,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
model.fit(X_train_LSTM, Y_train_LSTM, epochs = 60, batch_size = 32, shuffle=True)

Epoch 1/60
13/13 [==============================] - 10s 519ms/step - loss: 2.0809 - accuracy: 0.1172
Epoch 2/60
13/13 [==============================] - 7s 507ms/step - loss: 2.0746 - accuracy: 0.1322
Epoch 3/60
13/13 [==============================] - 7s 515ms/step - loss: 2.0636 - accuracy: 0.1496
Epoch 4/60
13/13 [==============================] - 7s 512ms/step - loss: 2.0574 - accuracy: 0.1796
Epoch 5/60
13/13 [==============================] - 7s 514ms/step - loss: 2.0425 - accuracy: 0.2195
Epoch 6/60
13/13 [==============================] - 7s 512ms/step - loss: 2.0089 - accuracy: 0.2344
Epoch 7/60
13/13 [==============================] - 7s 513ms/step - loss: 1.9503 - accuracy: 0.2594
Epoch 8/60
13/13 [==============================] - 7s 515ms/step - loss: 1.8771 - accuracy: 0.2668
Epoch 9/60
13/13 [==============================] - 7s 514ms/step - loss: 1.8228 - accuracy: 0.2743
Epoch 10/60
13/13 [==============================] - 7s 510ms/step - loss: 1.7826 - accuracy: 0.299

In [ ]:
loss, acc = model.evaluate(X_test_LSTM, Y_test_LSTM)
print()
print("Test accuracy = ", acc)

4/4 [==============================] - 2s 370ms/step - loss: 4.4854 - accuracy: 0.1250

Test accuracy =  0.125
